In [1]:
# Redo of the graph project Nov 2016
# This project will scrape a New York Social Diary website and identify divs with
# photos and photo captions. Parsing the captions will yield a social network graph 
# of the New York Social Elite.

# Paln for the graph visulization will be done with Tableau or networkx -> graph_tool or gephi

In [240]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 2 * matplotlib.rcParams['savefig.dpi']
import requests
from bs4 import BeautifulSoup
import urllib2
import re
from xml.sax.saxutils import escape
from HTMLParser import HTMLParser
import numpy as np
from multiprocessing import Pool
import networkx as nx
import pickle
import dill
import time
import itertools
from itertools import chain
import csv
#import nltk
#from nltk.tag import pos_tag

In [3]:
# create a subclass and override the handler methods
class MyHTMLParser_test(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print "Encountered a start tag:", tag
        #for name, value in attrs:
        #print value
    def handle_endtag(self, tag):
        #return
        print "Encountered an end tag :", tag

    def handle_data(self, data):
        #return
        print "Encountered some data  :", data

In [4]:
# create a subclass and override the handler methods
class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        if tag == "a":
            #print "Encountered a start tag:", tag
            for name, value in attrs:
                print value
    def handle_endtag(self, tag):
        return
        #print "Encountered an end tag :", tag

    def handle_data(self, data):
        return
        #print "Encountered some data  :", data

In [5]:
# redefine parser
class MyHTMLParser_2(HTMLParser):
    def handle_starttag(self, tag, attrs):
        if tag == "a":
            #print "Encountered a start tag:", tag
            for name, value in attrs:
                self.data = value 
                #this overwrites the data aspect which is our captions
                #print value
                #print name
                #print self.data
    def handle_endtag(self, tag):
        return
#print "Encountered an end tag :", tag

    def handle_data(self, data):
        self.data = self.data
        return
#       print "Encountered some data  :", data

In [6]:
# Here we go
# example album is 
# http://www.newyorksocialdiary.com/party-pictures/2016/celebrating-the-neighborhood

# The archive is a number of pages

# 0th http://www.newyorksocialdiary.com/party-pictures
# 1st http://www.newyorksocialdiary.com/party-pictures?page=1
# 28th and last is http://www.newyorksocialdiary.com/party-pictures?page=27

# so there are 28 pages

In [7]:
url = "http://www.newyorksocialdiary.com/party-pictures"

In [8]:
response = requests.get(url, params={"page": 0})
print response.url
response.text[:1000] + "..."

http://www.newyorksocialdiary.com/party-pictures?page=0


u'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n  <!--[if lte IE 6]><html class="no-js ie lt-ie9 lt-ie8 lt-ie7" lang="en" dir="ltr"><![endif]-->\n  <!--[if (IE 7)&(!IEMobile)]><html class="no-js ie lt-ie9 lt-ie8" lang="en" dir="ltr"><![endif]-->\n  <!--[if IE 8]><html class="no-js ie lt-ie9" lang="en" dir="ltr"><![endif]-->\n  <!--[if (gte IE 9)|(gt IEMobile 7)]><html class="no-js ie" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><![endif]-->\n  <!--[if !IE]><!--><html class="no-js" lang="en" dir="ltr" prefix="fb: http://ogp.me/ns/fb# og: http://ogp.me/ns# article: http://ogp.me/ns/article# book: http://ogp.me/ns/book# profile: http://ogp.me/ns/profile# video: http://ogp.me/ns/video# product: http://ogp.me/ns/product#"><!--<![endif]-->\n<he

In [9]:
# there is page zero remember 
# http://www.newyorksocialdiary.com/party-pictures?page=0
# page 29 does not exist currently byt 28 does and looks to be empty

In [10]:
soup = BeautifulSoup(urllib2.urlopen(response.url), "lxml") # lxml gets rid of the parser choice warning

In [11]:
parent_div = soup.find('div', attrs={'class': 'view-content'}) #Find (at most) *one*

In [12]:
parent_div

<div class="view-content">\n<div class="views-row views-row-1 views-row-odd views-row-first">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/celebrating-the-neighborhood">Celebrating the Neighborhood</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Monday, November 14, 2016</span> </span> </div>\n<div class="views-row views-row-2 views-row-even">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/a-rarity-these-days">A rarity these days</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Tuesday, November 8, 2016</span> </span> </div>\n<div class="views-row views-row-3 views-row-odd">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/fall-fetes">Fall Fetes</a></span> </span>\n<span class="views-field views-field-created"> <span class="fi

In [13]:
# img_divs = parent_div.find_all('div', attrs={'class':'technology'})  #Find *all* # fails returns []

In [14]:
link_divs = parent_div.find_all('div', attrs={'class':'views-row'})  #Find *all*
len(link_divs) # number of albums on page 0

50

In [15]:
link_divs # where we have href links with album names!

[<div class="views-row views-row-1 views-row-odd views-row-first">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/celebrating-the-neighborhood">Celebrating the Neighborhood</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Monday, November 14, 2016</span> </span> </div>,
 <div class="views-row views-row-2 views-row-even">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/a-rarity-these-days">A rarity these days</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Tuesday, November 8, 2016</span> </span> </div>,
 <div class="views-row views-row-3 views-row-odd">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/fall-fetes">Fall Fetes</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Monday, Nove

In [16]:
# 1st album when I do this is in here - Celebrating the neighborhood Monday Nov 14th
link_divs[0]

<div class="views-row views-row-1 views-row-odd views-row-first">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/celebrating-the-neighborhood">Celebrating the Neighborhood</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Monday, November 14, 2016</span> </span> </div>

In [17]:
# Some ways to detect href links I tried - but then I found MyHTMLParser()...

In [18]:
def detect_links(html):
    links = re.findall(r'<\s*a [^>]*href="([^"]*)"[^>]*>(.*?)</a', html)
    return ((href.strip(), re.sub('<[^>]*>', '', title.strip())) for href, title in links)

def detect_links2(html):
    soup = BeautifulSoup(html, "lxml")
    links = soup.find_all('a')
    return ((escape(link.get('href')), link.text.strip()) for link in links)

In [19]:
detect_links('link_divs[0]')

<generator object <genexpr> at 0x7efcf5a6d3c0>

In [20]:
detect_links2('link_divs[0]')

<generator object <genexpr> at 0x7efcf5a6db90>

In [21]:
# test call yields data now href value
parser_tester = MyHTMLParser_test()
parser = MyHTMLParser()
parser_2 = MyHTMLParser_2()

In [22]:
parser_tester.feed('<p><a href="http://www.quackit.com/html/tutorial/html_links.cfm">Example Link</a></p>')

Encountered a start tag: p
Encountered a start tag: a
Encountered some data  : Example Link
Encountered an end tag : a
Encountered an end tag : p


In [23]:
parser_2.feed('<p><a href="http://www.quackit.com/html/tutorial/html_links.cfm">Example Link</a></p>')

In [24]:
parser_2.data

'http://www.quackit.com/html/tutorial/html_links.cfm'

In [25]:
# example of how simple parser can be
parser.feed('<p><a href="http://www.quackit.com/html/tutorial/html_links.cfm">Example Link</a></p>')

http://www.quackit.com/html/tutorial/html_links.cfm


In [26]:
parser.feed('<div class="views-row views-row-1 views-row-odd views-row-first">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/summer-swan-song">Summer swan song</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Tuesday, September 6, 2016</span> </span> </div>')

/party-pictures/2016/summer-swan-song


In [27]:
parser_2.feed('<div class="views-row views-row-1 views-row-odd views-row-first">\n<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/celebrating-the-neighborhood">Celebrating the Neighborhood</a></span> </span>\n<span class="views-field views-field-created"> <span class="field-content">Monday, November 14, 2016</span> </span> </div>')

In [28]:
parser_2.data

'/party-pictures/2016/celebrating-the-neighborhood'

In [29]:
div_list = []
for i in range(0,len(link_divs)):
    div_list.append(str(link_divs[i]))        # div_list with album titles
    print link_divs[i]

<div class="views-row views-row-1 views-row-odd views-row-first">
<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/celebrating-the-neighborhood">Celebrating the Neighborhood</a></span> </span>
<span class="views-field views-field-created"> <span class="field-content">Monday, November 14, 2016</span> </span> </div>
<div class="views-row views-row-2 views-row-even">
<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/a-rarity-these-days">A rarity these days</a></span> </span>
<span class="views-field views-field-created"> <span class="field-content">Tuesday, November 8, 2016</span> </span> </div>
<div class="views-row views-row-3 views-row-odd">
<span class="views-field views-field-title"> <span class="field-content"><a href="/party-pictures/2016/fall-fetes">Fall Fetes</a></span> </span>
<span class="views-field views-field-created"> <span class="field-content">Monday, November 7, 201

In [30]:
# print all album names so far
for i in range(0,len(div_list)):
    parser.feed(div_list[i])

print "There are "+str(i+1)+" albums in total"

/party-pictures/2016/celebrating-the-neighborhood
/party-pictures/2016/a-rarity-these-days
/party-pictures/2016/fall-fetes
/party-pictures/2016/distinguished-service
/party-pictures/2016/the-arts
/party-pictures/2016/arts-and-education
/party-pictures/2016/trailblazers
/party-pictures/2016/lighting-up-lives
/party-pictures/2016/an-extra-boost
/party-pictures/2016/when-museums-are-built
/party-pictures/2016/fulfillment-of-agenda
/party-pictures/2016/a-look-around-the-neighborhood
/party-pictures/2016/new-york-after-dark
/party-pictures/2016/fact-vs-fiction
/party-pictures/2016/fall-fun
/party-pictures/2016/lessons-learned
/party-pictures/2016/championing-the-makers
/party-pictures/2016/same-sky
/party-pictures/2016/new-yorkers-for-children-celebrates-20
/party-pictures/2016/artistry-of-fashion
/party-pictures/2016/summer-swan-song
/party-pictures/2016/the-grand-voyage
/party-pictures/2016/classic-moments
/party-pictures/2016/summer-fun-and-fashion
/party-pictures/2016/the-7th-annual-apo

In [31]:
# Now we loop over the pages to get all the albums on the site...
# sub-select from a data range
# extract all captions
# extract captions with less than 250 characters - longer captions are not just names...
# make a network graph!

In [32]:
master_list = []
for i in range(3,5): #28): # old project date range
#for i in range(0,28): # very dirty getting rid of date stuff
    response = requests.get(url, params={"page": i})
    #wait time
    #print response.url
    soup = BeautifulSoup(urllib2.urlopen(response.url),"lxml")
    parent_div = soup.find('div', attrs={'class': 'view-content'}) #Find (at most) *one*
    link_divs = parent_div.find_all('div', attrs={'class':'views-row'})  #Find *all*
    #print len(link_divs)
    #print link_divs[0]
    #break
    new_list = []
    for j in range(0,len(link_divs)):
        new_list.append(str(link_divs[j]))
        parser_2.feed(new_list[j])
        #print parser_2.data
        #break
        master_list.append(parser_2.data)

    time.sleep(1)

In [33]:
print len(master_list)

100


In [34]:
master_list[:36] # so with normal parser we get none with the _2 parser we get null '' string! ? why ?

['/party-pictures/2015/madoo-in-manhattan-pmp-in-palm-beach',
 '/party-pictures/2015/love-heals',
 '/party-pictures/2015/citymeals-on-wheels-mount-sinai-hospital-un-women-for-peace-and-the-red-cross',
 '/party-pictures/2015/ripa-eight-over-eighty-trees-new-york-and-fabulous-at-any-age',
 '/party-pictures/2015/bunny-hop-the-boys-club-old-bags-and-more',
 '/party-pictures/2015/sab-the-jewish-museum-roundabout-theatre-and-faces',
 '/party-pictures/2015/adaa-art-show-bronx-museum-the-china-arts-foundation-international-and-the-palm',
 '/party-pictures/2015/the-60th-anniversary-of-the-viennese-opera-ball-and-mcnys-directors-council',
 '/party-pictures/2015/the-new-york-philharmonic-the-new-york-botanical-garden-longhouse-reserve-and',
 '/party-pictures/2015/mission-accomplished',
 '/party-pictures/2015/philanthropic-endeavors',
 '/party-pictures/2015/dining-with-the-divas',
 '/party-pictures/2015/fielding-dreams',
 '/party-pictures/2015/generosity-leadership',
 '/party-pictures/2015/young-n

In [35]:
type(master_list[0])

str

In [36]:
new_list2014 = master_list[35:]
len(new_list2014)

65

In [37]:
new_list2014[0] # we want to start with '/party-pictures/2014/gala-guests' - number increases as we move into future

'/party-pictures/2014/gala-guests'

In [46]:
# You know what this does, right?
# It gets all the div data in one swoop

def myhtmlcall(link):
    url =  "http://www.newyorksocialdiary.com" + link
    return requests.get(url).text
    
p = Pool(8)
newobj  = p.map(myhtmlcall, new_list2014)


# save to pickle
with open("all_div_all_captions.pickle", "wb") as file:
    pickle.dump(newobj, file)
    
# read in as n = pickle.load(open('file.pickle', 'rb'))

In [47]:
# only print this if len(new_list2014) < 100 or
newobj[0][0:100]

u'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n '

In [48]:
oldobj = pickle.load(open('all_div_all_captions.pickle', 'rb'))

In [49]:
oldobj[0][0:100]

u'<!DOCTYPE html>\n  <!--[if IEMobile 7]><html class="no-js ie iem7" lang="en" dir="ltr"><![endif]-->\n '

In [55]:
#BAD# create a subclass and override the handler methods
class MyHTMLParser3(HTMLParser):
    def handle_starttag(self, tag, attrs):
        if tag == "a":
            #print "Encountered a start tag:", tag
            for name, value in attrs:
                self.data = value # this overwrites the data aspect which is our captions
                #print value
    def handle_endtag(self, tag):
        return
        #print "Encountered an end tag :", tag##

    def handle_data(self, data):
        self.data = data
        
        #print "Encountered some data  :", data
        

In [56]:
parser2 = MyHTMLParser_2()
parser3 = MyHTMLParser3()

In [61]:
caption_div = []
caption_indx = []
master_caps = []
master_caps2 = []
for i in range(0,len(newobj)):
    soup = BeautifulSoup(newobj[i],"lxml")
    # this is the crucial step - this is where you miss captions!
    caption_divs_main = soup.find_all('font', attrs={'face': 'Verdana, Arial, Helvetica, sans-serif'}) #Find all* # 5385
    ####caption_divs1 = soup.find_all('div', attrs={'class': 'photocaption'}) #Find all* # 73113
    caption_divs2 = soup.find_all("div", {'align': 'center', 'class': 'photocaption'}, text=True)
    caption_divs3 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption'}, text=True)
    caption_divs4 = soup.find_all("td", {'valign': 'top', 'class': 'photocaption', 'style': 'background-color:#faf9ee'}, text=True)
    ###caption_divs = soup.find_all("font", {'size': 1, 'face': 'Verdana, Arial, Helvetica, sans-serif'}, text=True) #5301
    caption_divs = caption_divs_main + caption_divs2 + caption_divs3 + caption_divs4
    #  print caption_divs_main # useful for debugging
    #  caption_divs = caption_divs_main
    caption_indx.append(len(caption_divs))
    for j in range(0,len(caption_divs)):
        master_caps.append(caption_divs[j])
        #print caption_divs[j]
        parser3.feed(str(caption_divs[j]))
        master_caps2.append(parser3.data)
        #parser2.feed(str(caption_divs[j]))
        #master_caps2.append(parser2.data)
print len(master_caps), sum(caption_indx), len(master_caps2)

5631 5631 5631


In [65]:
master_caps[0:2]

[<div align="center" class="photocaption">The scene at   IDEAL School &amp; Academy\u2019s  10th\xa0Annual Gala.</div>,
 <div align="center" class="photocaption"> Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino </div>]

In [66]:
master_caps2[0:2]

[' Academy\xe2\x80\x99s  10th\xc2\xa0Annual Gala.',
 ' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino ']

In [67]:
print str(master_caps2[1345])

 Congresswoman Carolyn Maloney and Manhattan Borough President Gale Brewer 


In [68]:
#sum = 0
list250char = []
#print len(master_caps2[1345])
for i in range(0,len(master_caps2)):
    if len(str(master_caps2[i])) > 249: # was 250
        pass
        #print i, str(master_caps2[i])
        #sum += 1
    else:
        list250char.append(master_caps2[i])

In [69]:
list250char

[' Academy\xe2\x80\x99s  10th\xc2\xa0Annual Gala.',
 ' Les Lieberman, Barri Lieberman, Isabel Kallman, Trish Iervolino, and Ron Iervolino ',
 ' Chuck Grodin ',
 ' Diana Rosario, Ali Sussman, Sarah Boll, Jen Zaleski, Alysse Brennan, and Lindsay Macbeth ',
 ' Kelly  and Tom Murro ',
 ' Udo Spreitzenbarth ',
 ' Ron Iervolino, Trish Iervolino, Russ Middleton, and Lisa Middleton ',
 ' Barbara Loughlin, Dr. Gerald Loughlin, and Debbie Gelston ',
 ' Julianne Michelle ',
 ' Heather Robinson, Kiwan Nichols, Jimmy Nichols, Melanie Carbone, and Nancy Brown ',
 ' Bill Mack and Les Lieberman ',
 ' Dr. David Lyden and Patricia Sorenson ',
 ' Jimmy Cayne, Vince Tese, and Pat Cayne ',
 ' Stuart Oran, Les Lieberman, and Hilary Oran ',
 ' Vince Tese and Chuck Grodin ',
 ' Dwight Gooden and Les Lieberman ',
 ' Dr. Amy Cunningham-Bussel, Ray Mirra, and Dr. Tyler Janovitz ',
 ' Dan Shedrick and Samara Heafitz ',
 ' Cass and Jason Adelman ',
 ' Bart Scott and Mark Laplander ',
 ' Mitch Rubin, Audra Zuckerma

In [71]:
len(list250char)

5614

In [72]:
# write out captions
thefile = open('NOV16_az_captions.txt', 'w') # was test_az...
for item in list250char:
  thefile.write("%s\n" % item)

# save to pickle
with open("NOV16_az_captions.pickle", "wb") as file:
    pickle.dump(list250char, file)


In [ ]:
# In part 2 we extract names from captions
# and make a social graph, calculate page rank etc
# In part 3 we explore visualization tools for social graph networks